In [22]:
from uproot_module import Data
import uproot
import awkward
import pandas as pd

In [6]:
SIGNAL_PATH = "/home/santiago/VBF_DMSimp_spin0_EWKExcluded/Events/run_18/DMSimpSpin0_MY5000_MX1000_07042020.root"
TREE_NAME = "Delphes"

pt = ("Jet.PT[%d]"%(i) for i in range(2))
phi = ("Jet.Phi[%d]"%(i) for i in range(2))
d = Data(*(*pt,*phi,"MissingET.MET","MissingET.Phi"))
d.setSignal(SIGNAL_PATH, TREE_NAME)
d.dataframe.head()

Jet.PT[[0, 1]]
Jet.Phi[[0, 1]]


,Jet.PT,Jet.Phi,MissingET.MET,MissingET.Phi
0,"[2611.701, 2441.1091, 27.122381, 24.460632]","[-1.361408, 1.7731673, -1.8224834, 0.4630849]",198.476883,1.941587
1,"[2121.402, 978.5685, 689.59515, 127.36351, 54....","[0.035726447, 2.9122832, -2.657105, -2.9357722...",472.074036,3.070084
2,"[2065.1433, 1989.5405, 185.70207, 33.396263, 3...","[-0.76010954, 2.4621053, 0.5589036, -2.969895,...",136.096527,2.243740
3,"[1088.2563, 884.0749, 173.26927, 105.40972]","[0.4684148, -2.658156, -2.6574259, -2.0437043]",111.293030,1.529968
4,"[2405.0735, 2305.2903, 203.49843, 67.99351, 37...","[2.937418, -0.23584753, 2.1395578, -1.329002, ...",180.646027,-0.234490


In [4]:
dfPT = d.dataframe["Jet.PT"]

In [33]:
sig = uproot.open(SIGNAL_PATH)[TREE_NAME]
arr = sig.arrays(branches=["Jet.PT"], outputtype=pd.DataFrame)
arr.unstack()

        entry
Jet.PT  0              [2611.701, 2441.1091, 27.122381, 24.460632]
        1        [2121.402, 978.5685, 689.59515, 127.36351, 54....
        2        [2065.1433, 1989.5405, 185.70207, 33.396263, 3...
        3              [1088.2563, 884.0749, 173.26927, 105.40972]
        4        [2405.0735, 2305.2903, 203.49843, 67.99351, 37...
                                       ...                        
        34074    [2582.4846, 2115.521, 535.0259, 221.5823, 64.4...
        34075    [2304.3674, 2259.9983, 57.157444, 30.430975, 2...
        34076                    [1492.3352, 1305.7035, 46.217793]
        34077    [2205.585, 2126.9424, 215.92505, 140.37036, 93...
        34078         [1612.1184, 1224.5667, 257.48444, 39.757782]
Length: 34079, dtype: object

In [44]:
df = sig.pandas.df("Jet.PT").unstack()
df["Jet.PT"].columns

Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], dtype='int64', name='subentry')